# **Solar Pro2 Prompting Handbook**

## **1. Overview**

solar-pro2 is a reasoning model optimized for high-performance inference and diverse task execution. This guide presents a systematic prompting methodology to help developers maximize the enhanced capabilities of solar-pro2. Since solar-pro2 is highly sensitive to clear and structured instructions, effective prompt design directly impacts performance. Along with existing general best practices, a prompt approach tailored to solar-pro2's characteristics is necessary.

All examples in this guide consist of executable code that can be immediately tested and verified. Experience the performance by directly running the prompt examples in each section. Please follow these steps before running this notebook:

1. **API Key Issuance**: Sign up at [Upstage Console](https://console.upstage.ai/api-keys?api=chat) and obtain your API key
2. **Environment Variable Setup**: Set the obtained API key as an environment variable or input it directly
3. **Package Installation**: Install the required packages and complete the API setup

**Note**: Please be careful not to expose your API key externally!


In [1]:
# Install required packages
import subprocess
import sys

def install_package(package):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ {package} installation completed")
    except subprocess.CalledProcessError:
        print(f"❌ {package} installation failed")

# Install required packages
packages = [
    "openai>=1.0.0",
    "requests",
    "python-dotenv"
]

print("📦 Installing required packages...")
for package in packages:
    install_package(package)


📦 Installing required packages...



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


✅ openai>=1.0.0 installation completed



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


✅ requests installation completed
✅ python-dotenv installation completed



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import openai
from openai import OpenAI
import getpass
from dotenv import load_dotenv

# Load .env file if it exists
load_dotenv()


def setup_api_key():
    """Function to setup API key"""
    api_key = os.getenv("UPSTAGE_API_KEY")

    if not api_key:
        print("🔑 Upstage API key is not set in environment variables.")
        print("Please input the API key directly or set it in the UPSTAGE_API_KEY environment variable.")
        print("\nOption 1: Direct input (valid only for current session)")
        api_key = getpass.getpass("Enter your Upstage API key: ")

    if api_key:
        os.environ["UPSTAGE_API_KEY"] = api_key
        print("✅ API key has been set!")
        return api_key
    else:
        print("❌ API key has not been set.")
        return None


# Setup API key
api_key = setup_api_key()

# Setup OpenAI client (using Upstage API endpoint)
if api_key:
    client = OpenAI(api_key=api_key, base_url="https://api.upstage.ai/v1/solar")
    print("solar-pro2 API is ready!")
else:
    print("⚠️ Unable to setup API client. Please check your API key.")

🔑 Upstage API key is not set in environment variables.
Please input the API key directly or set it in the UPSTAGE_API_KEY environment variable.

Option 1: Direct input (valid only for current session)
✅ API key has been set!
solar-pro2 API is ready!


### **1.1 What is Reasoning?**

solar-pro2 improves performance by allowing the model to generate an internal "chain of thought" before responding. Through this, the model can:
- Break down complex questions into smaller steps
- Decompose complex queries into logical subtasks
- Accurately perform complex and numerous instructions

Leverage the model's structured chain of thought process for complex multi-step tasks (e.g., document analysis, code generation, complex logical problems).


---

## **2. Parameter Usage**

### **2.1 Basic Parameters**

#### 2.1.1 Temperature

| Value Range | Recommended Use | Characteristics |
|-------------|-----------------|-----------------|
| **0.0-0.3** | Translation, summarization, analysis | Tasks where consistency is important |
| **0.4-0.7** | General conversation, explanation | Balanced creativity and accuracy |
| **0.8-2.0** | Writing, brainstorming | Creative work |

#### 2.1.2 Max Tokens

| Token Range | Recommended Use |
|-------------|-----------------|
| **50-200** | Keyword extraction, classification |
| **200-800** | Summarization, explanation |
| **1000+** | Detailed analysis, long-form writing |

#### 2.1.3 Top_p

| Value Range | Recommended Use |
|-------------|-----------------|
| **0.1-0.5** | Technical documents, accurate information |
| **0.6-0.8** | General purpose |
| **0.9-1.0** | Creative work |


### **2.2 Reasoning Effort**

The solar-pro2 model is a hybrid model that can select reasoning mode based on task type. The `reasoning_effort` parameter allows two settings: low/high.

#### 2.2.1 Low Reasoning Effort

This is a mode set with reasoning_effort="low" for general tasks that do not require complex reasoning processes. This mode offers fast processing speed and cost efficiency, making it suitable for high-volume simple tasks or services requiring real-time responses.

**Use Cases:**
- General Q&A: Fact verification, definition explanation
- Simple translation: Direct translation, basic language conversion
- Information extraction: Extracting specific data from text
- Format conversion: Data format changes

**Example: Information Extraction**


In [3]:
response = client.chat.completions.create(
    model="solar-pro2",
    messages=[
        {
            "role": "system",
            "content": """Extract the following information from the [Text] I give and return as a list without any explanation:
- Person names
- Dates
- Locations"""
        },
        {
            "role": "user",
            "content": """Text: "John Smith visited Paris on March 15, 2024, and met with Sarah Johnson at the Louvre Museum.\""""
        }
    ],
    max_tokens=300,
    temperature=0.2,
    reasoning_effort="low"
)

print(response.choices[0].message.content)

- Person names: John Smith, Sarah Johnson
- Dates: March 15, 2024
- Locations: Paris, Louvre Museum


#### 2.2.2 High Reasoning Effort

Setting reasoning_effort="high" allows you to utilize the model's reasoning mode. This mode ensures that the model goes through a reasoning process before generating an answer, optimized for tasks requiring complex analysis and structured thinking. In this mode, the model systematically breaks down problems step by step and performs logical verification at each stage to generate more accurate and reliable results.

**Use Cases:**
- Mathematical problem solving: Multi-step calculations, proofs
- Logical reasoning: Syllogism, causal relationship analysis
- Complex analysis: Comprehensive review from multiple perspectives
- Strategy development: Planning considering various factors

**Example: Legal Case Entity Extraction**

Unlike the previous example, this is an information extraction task requiring complex calculations and logical verification. It goes beyond simply finding information from text to perform damage calculation through numerical computation and analyze discrepancies with claimed amounts.

In [ ]:
response = client.chat.completions.create(
   model="solar-pro2",
   messages=[
       {
           "role": "system",
           "content": """Extract property dispute details from the [Text] and return as lists only:
- Parties: [names and roles]
- Property details: [location, size, values]
- Damage calculations: [repair costs + lost rent with totals]
- Timeline: [key dates and duration of issues]
- Analysis: [verify if calculated total matches claimed amount, note any discrepancies]"""
       },
       {
           "role": "user",
           "content": """Text: "Landlord Sarah Kim is suing tenant Mike Johnson for $8,500 in damages to her downtown apartment. The lease shows monthly rent of $2,200. Johnson caused water damage in March 2024 requiring $3,500 in repairs. He also broke the lease early, leaving 2.5 months of unpaid rent before the July 2024 lease end date.\""""
       }
   ],
   max_tokens=8000, # Sufficient token count setting for reasoning
   temperature=0.2,
   reasoning_effort="high"
)

print(response.choices[0].message.content)

- **Parties**:  
  - Landlord: Sarah Kim  
  - Tenant: Mike Johnson  

- **Property details**:  
  - Location: Downtown apartment  
  - Size: N/A  
  - Values: Monthly rent $2,200 (lease term unspecified)  

- **Damage calculations**:  
  - Repair costs: $3,500 (water damage)  
  - Lost rent: 2.5 months × $2,200 = **$5,500**  
  - **Total claimed**: $8,500  
  - **Calculated total**: $3,500 + $5,500 = **$9,000**  

- **Timeline**:  
  - March 2024: Water damage occurred  
  - July 2024: Lease end date (tenant left 2.5 months early)  
  - Duration of unpaid rent: 2.5 months  

- **Analysis**:  
  - Calculated total ($9,000) **does not match** claimed amount ($8,500).  
  - Discrepancy of **$500** unaccounted for in the claim.


The reasoning process includes the following steps, leading to more accurate and higher-quality answers. When using reasoning mode, it is **recommended to set `max_token` to 4000 or higher** to include sufficient reasoning processes for deriving answers.

| Reasoning Step | Example of Model's Reasoning Content |
|----------------|-------------------------------------|
| **Step-by-step planning and task breakdown** | "First, I'll identify the parties involved. The landlord is Sarah Kim and the tenant is Mike Johnson. Their roles are clear from the text." |
| **Sequential information processing** | "Next, property details. The text mentions a downtown apartment, but there's no info on size or specific values beyond rent." |
| **Mathematical calculation and verification** | "For damage calculations, the repair costs are $3,500 from water damage. The monthly rent is $2,200, so 2.5 months would be $5,500." "Let me double-check. 2.5 months at $2,200 is 2.5 * 2200 = 5,500. Plus 3,500 repairs equals 9,000." |
| **Identifying reasoning contradictions and cause analysis** | "But the landlord is suing for $8,500. There's a discrepancy here. The total calculated is $9,000, but the claim is $8,500." "Maybe the landlord is not claiming the full 2.5 months? Need to mention this discrepancy." |
| **Final verification and structuring** | "Make sure each section is clear and only includes the information from the text." "Alright, putting it all together now." |

#### 2.2.3 High Reasoning Effort: When to Choose?

**Basic Strategy:** We recommend using reasoning_effort="high" only when quality is absolutely critical, after first understanding the nature of the task.

*Speed and output tokens may vary depending on task type.

| Category | reasoning_effort="high" | reasoning_effort="low" |
|----------|------------------------|----------------------|
| Processing Speed | Relatively slow | Can process about 3x faster |
| Output Tokens | Includes reasoning process tokens | Can reduce output tokens by approximately 70% |

Set appropriate parameters through these questions:

1) Quality vs Efficiency Priority
- Is result accuracy absolutely critical? → reasoning_effort="high"
- Are processing speed and cost more important? → reasoning_effort="low"

2) Task Complexity Assessment
- Do you need multi-step logical thinking? → reasoning_effort="high"
- Is it simple information provision or conversion work? → reasoning_effort="low"

3) Usage Environment Consideration
- Is it a service that needs real-time responses? → reasoning_effort="low"


---

## **3. Prompt Structure Design**

### **3.1 Core Elements**

Effective prompts consist of the following elements. Each element directly affects the model's comprehension and output quality, so they must be carefully designed according to the task characteristics:

* **Role**: Define the model's expertise and perspective
   * Activate domain-specific knowledge
   * Set response tone and style
* **Task**: Clear definition of the work to be performed
   * **Instructions**: Specific execution methods
   * **Constraints**: Rules and limitations to be followed
* **Output Format**: Define output structure and style
   * **Samples**: Concrete examples of expected output
* **User Input**: Actual data to be processed
   * **Contexts**: Background information needed for task execution
   * **Query**: Specific requests

### **3.2 Structural Combination Patterns**

Prompt structure should be combined differently according to task complexity and requirements. Since solar-pro2 understands structured information well, appropriate structure selection is key to performance improvement.

#### 3.2.1 Basic Combination
Suitable for simple tasks or general Q&A. Used when quick responses are needed and there are no complex constraints.

**Structure:** Role → Instructions → Query

#### 3.2.2 Fixed Output Format
Used when output in a predetermined format is required. Suitable for API responses, data transformation, structured report writing, etc.

**Structure:** Role → Instructions → Constraints → Samples → Query

#### 3.2.3 RAG (Retrieval-Augmented Generation)
Used for answer generation utilizing external information. Suitable for document analysis, information retrieval-based responses, knowledge base utilization, etc.

**Structure:** Contexts → Role → Instructions → Query


### **3.3 Importance of Element Placement Order**

The order of elements within prompts significantly affects the model's output tendencies, so placing important information in appropriate positions within the prompt is crucial.

Below is the general element placement order, but you should compare and analyze performance differences by changing the positions between elements, even for the same elements, or by changing positions between system prompts and user prompts. Through such systematic A/B testing, you can discover prompt structures optimized for specific tasks.

System Prompt
1. **Context Information**: Place at the very beginning to help understand the overall situation
2. **Role Definition**: Place after context to set the response perspective
3. **Specific Instructions**: Present detailed execution methods after role definition
4. **Constraints and Format**: Place immediately after instructions to manage output quality
5. **Examples**: Improve comprehension with concrete examples after format explanation

User Prompt
1. Specific tasks
2. User input data

### **3.4 Structured Formats**

#### 3.4.1 Markdown
Use markdown headings for main sections and subsections (including deeper hierarchies up to H4+). Use numbering or bullet points as needed.


In [5]:
response = client.chat.completions.create(
    model="solar-pro2",
    messages=[
        {
            "role": "system",
            "content": """# Role
You are a meeting organizer specializing in team communications.

## Instructions  
Create structured meeting agendas with clear time management and actionable outcomes.

#### Example Format
**Topic Name [X min]**
- Discussion point 1
- Discussion point 2

Generate a clear meeting agenda following this structure."""
        },
        {
            "role": "user",
            "content": """Create a 20-minute standup meeting agenda covering project updates and blockers."""
        }
    ],
    max_tokens=2000,
    temperature=0.7,
    reasoning_effort="high"
)

print(response.choices[0].message.content)


**Team Standup Meeting Agenda [20 min]**  

**1. Quick Check-in [2 min]**  
- Confirm agenda and time constraints  
- Brief team status check (e.g., "Ready to proceed?")  

**2. Project Updates (Rotating Order) [8 min]**  
- **Each member (2 mins):**  
  - *Key accomplishments* since last standup  
  - *Priorities for the day*  
  - *Progress toward sprint/milestone goals*  

**3. Blockers & Dependencies [5 min]**  
- Identify unresolved blockers (e.g., technical issues, resource gaps)  
- Discuss cross-team dependencies or external delays  
- Assign immediate follow-up owners for resolution (e.g., "X will email Y by EOD")  

**4. Next Steps & Accountabilities [5 min]**  
- Clarify action items from blockers (e.g., "Dev team to prioritize API fix today")  
- Assign deadlines and responsible parties for follow-ups  
- Confirm individual daily goals and sprint alignment  

**Adj


#### 3.4.2 XML Tags

Use when you need to precisely wrap sections including start and end tags. You can add metadata to tags for additional context, and create nested structures by placing new tags within the same tag.

The example below is a prompt for report writing tasks. The structure is clearly expressed through XML tags, and character limits for each structure are expressed through metadata.


In [6]:
response = client.chat.completions.create(
    model="solar-pro2",
    messages=[
        {
            "role": "system",
            "content": """<role>You are a business analyst.</role>
<instructions>
 <rule1>Generate a structured report with plain text following the hierarchy and 'max_chars' limits</rule1>
 <rule2>Count characters in each section and ensure none exceed their limits. If any section exceeds the limit, revise it to fit.</rule2>
 <rule3>Do not include character counts in your response.</rule3>
</instructions>
 <report_structure>
 <summary max_chars="100">brief overview</summary>
 <key_finding max_chars="100">main insight</key_finding>
 <recommendation max_chars="100">suggested action</recommendation>
</report_structure>"""
        },
        {
            "role": "user",
            "content": """Write a report about AI industry trends."""
        }
    ],
    max_tokens=4000,
    temperature=0.3,
    reasoning_effort="high"
)

print(response.choices[0].message.content)


**Summary**  
AI industry growth driven by generative AI, sector integration, and ethical focus.  

**Key Finding**  
Generative AI leads innovation, transforming industries with advanced NLP and automation.  

**Recommendation**  
Invest in AI talent, ethical frameworks, and cross-industry partnerships to stay competitive.


---

## **4. Prompt Methods**

### **4.1 Role**

Role definition has a decisive impact on the model's response quality and consistency. Since solar-pro2 applies domain-specific knowledge and perspectives very well according to roles, specific and clear role setting is important.

#### 4.1.1 Domain-Specialized Experts

For tasks requiring deep knowledge and experience in specialized fields, assign actual job categories from that field to the model.
The example below activates medical expertise by assigning a specific role as an emergency medicine specialist. By specifying 15 years of experience and detailed specialty areas, the model is guided to apply systematic diagnostic approaches and clinical protocols in emergency situations.


In [7]:
response = client.chat.completions.create(
    model="solar-pro2",
    messages=[
        {
            "role": "system",
            "content": """# Role
You are a board-certified emergency medicine physician with 15 years of clinical experience. You specialize in rapid assessment, differential diagnosis, and emergency treatment protocols.

# Instructions
Provide brief medical assessment:
- Top 3 differential diagnoses
- Immediate action needed."""
        },
        {
            "role": "user",
            "content": """A 45-year-old patient presents with chest pain, shortness of breath, and sweating. What's your initial assessment?"""
        }
    ],
    max_tokens=4000,
    temperature=0.3,
    reasoning_effort="high"
)

print(response.choices[0].message.content)


**Initial Assessment:**  
A 45-year-old with chest pain, shortness of breath, and diaphoresis requires urgent evaluation for life-threatening causes.  

### **Top 3 Differential Diagnoses:**  
1. **Acute Coronary Syndrome (ACS)/Myocardial Infarction (MI):**  
   - Classic presentation with chest pain, dyspnea, and diaphoresis. Risk factors (e.g., hypertension, smoking, diabetes) increase likelihood.  
   - **Key findings:** ECG changes (ST elevation/depression), elevated cardiac biomarkers (troponin).  

2. **Pulmonary Embolism (PE):**  
   - Sudden dyspnea, pleuritic or substernal chest pain, tachycardia. Consider in patients with immobility, hypercoagulable states, or unexplained hypoxia.  
   - **Key findings:** Hypoxia, tachycardia, elevated D-dimer (if pre-test probability is intermediate/low), CT pulmonary angiogram confirmation.  

3. **Aortic Dissection:**  
   - Severe, tearing chest/back pain, possible pulse deficits or blood pressure asymmetry. Hypertension is a major risk f

#### 4.1.2 Task Performance Specialists

Even for tasks where specific professions don't exist, you can create and assign virtual expert personas with specialized expertise for those tasks to provide context to the model.


In [8]:
response = client.chat.completions.create(
    model="solar-pro2",
    messages=[
        {
            "role": "system",
            "content": """# Role
You are a search optimization specialist with expertise in keyword extraction and information retrieval.

# Instructions
Extract 3 main keywords from the query and return them as a simple list without any explanation:
1. [primary keyword]
2. [secondary keyword] 
3. [related term]

Focus on the most important search terms for optimal information retrieval."""
        },
        {
            "role": "user",
            "content": """I need help with machine learning algorithms for natural language processing in Python."""
        }
    ],
    max_tokens=400,
    temperature=0.5,
    reasoning_effort="low"
)

print(response.choices[0].message.content)


1. machine learning algorithms  
2. natural language processing  
3. Python


### **4.2 Instructions**

Instructions are a key element that determines the success of tasks. Since solar-pro2 responds very well to clear and specific instructions, ambiguity must be removed and step-by-step detailed instructions must be provided.

#### 4.2.1 Requirements Breakdown

It's important to write requirements as specifically as possible for the model to understand. If task scenarios are diverse, you need to write separate guidelines to follow based on the type of User Input or Context. Instead of vague instructions, present measurable and actionable criteria to ensure consistent results and include detailed guidelines that can respond to various situations.

The prompt below is an example of breaking down title generation tasks by content type. For each type, specific pattern formulas, focus areas, tone, and character limits are specified to transform the qualitative requirement of "good title" into concrete criteria.


In [9]:
system_prompt = """# Role
You are a title optimization specialist for different content platforms.

# Instructions
Create titles based on content type:

## Educational Titles
- Pattern: "How to [skill]" or "[Number] Steps to [outcome]"
- Focus: Clear learning outcomes, instructional tone

## Blog Titles  
- Pattern: "[Number] [power word] [topic] [benefit]"
- Focus: Curiosity, engagement, personal benefit

## News Titles
- Pattern: "[Who] [action] [what] [impact]"
- Focus: Facts, immediacy, authority

# Constraints
- Title must be under 100 characters
- Return only the title matching the requested type."""

Check the results of putting three types of content into the user prompt according to the above prompt.

In [11]:
# Education 유형의 글을 user prompt에 넣었을 때
response_edu = client.chat.completions.create(
    model="solar-pro2",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "This comprehensive Python tutorial covers essential programming fundamentals for beginners. You'll learn how to declare and use variables to store data, create loops to repeat code efficiently, and write functions to organize your programs. Each concept includes hands-on examples and practice exercises to reinforce your understanding. By the end of this guide, you'll have the foundational skills needed to build your first Python applications."}
    ],
    max_tokens=200,
    temperature=0.4,
    reasoning_effort="low"
)
print("Educational Title:", response_edu.choices[0].message.content)

# Blog 유형의 글을 user prompt에 넣었을 때
response_blog = client.chat.completions.create(
    model="solar-pro2", 
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "Working from home during the holiday season presents unique productivity challenges. Family gatherings, increased distractions, and holiday stress can derail your focus. However, with strategic time-blocking techniques, you can maintain peak performance. Set clear boundaries with family members about work hours, create a dedicated workspace away from holiday activities, and use the Pomodoro technique to stay focused during short bursts. These proven strategies help remote workers maintain productivity even during the busiest holiday periods."}
    ],
    max_tokens=200,
    temperature=0.4,
    reasoning_effort="low"
)
print("Blog Title:", response_blog.choices[0].message.content)

# News 유형의 글을 user prompt에 넣었을 때
response_news = client.chat.completions.create(
    model="solar-pro2",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "Create a news title for: Meta Platforms announced today that it will lay off approximately 10,000 employees across multiple divisions, with engineering and product development teams bearing the brunt of the cuts. CEO Mark Zuckerberg cited economic uncertainty and the need to streamline operations as primary reasons for the workforce reduction. The layoffs represent about 13% of Meta's total workforce and are expected to be completed by the end of the quarter. This marks the company's second major round of layoffs in six months, following similar cuts in November 2022."}
    ],
    max_tokens=200,
    temperature=0.4,
    reasoning_effort="low"
)
print("News Title:", response_news.choices[0].message.content)

Educational Title: "How to Master Python Fundamentals: Beginner's Step-by-Step Guide"
Blog Title: 5 Proven Time-Blocking Hacks to Stay Productive Working From Home This Holiday Season
News Title: Meta Platforms Cuts 10,000 Jobs Amid Economic Uncertainty, Streamlining Push


#### 4.2.2 Presenting Specific Execution Methods

Guide accurate execution methods by breaking down complex and difficult tasks step by step.

The prompt below is an example of a translation task that converts English number units to Korean. It guides a systematic conversion process through identifying English number units (thousand, million, billion, trillion), presenting basic conversion formulas and compound number breakdown methods, and verifying mathematical accuracy.

In [17]:
response = client.chat.completions.create(
   model="solar-pro2",
   messages=[
       {
           "role": "system",
           "content": """# Role
You are an English-Korean business translator specializing in financial documents.

# Instructions
Translate English text to Korean with precise number unit conversion:

### English Number Units
- thousand → 천
- million → 백만 (or 100만)
- billion → 십억 (or 10억) 
- trillion → 조

### Trillion-level Numbers
Break down compound numbers: 15.2 trillion = 15조 2천억

### Million-level Numbers
- 32 million = 3천 2백만
- Pattern: X.Y million = X천 Y백만
- Example: 12.5 million = 1천 2백 5십만

### Format Requirements
- Use standard Korean business notation
- Convert currency appropriately (dollars to won if needed)
- Maintain formal business Korean tone

CRITICAL: Convert ALL English numbers accurately to Korean units. Return only the translated sentence."""
       },
       {
           "role": "user",
           "content": """TechCorporation achieved revenue of 8.3 trillion won this year, which represents an increase of 1.2 trillion won compared to the previous year. The AI division generated 2.7 trillion won, while the cloud division earned 350 billion won in revenue. Net profit recorded 1.8 trillion won, and 420 billion won was invested in research and development. The company hired 25,000 new employees with an average annual salary of 85 million won."""
       }
   ],
   max_tokens=4000,
   temperature=0.1,
   reasoning_effort="high"
)

print(response.choices[0].message.content)

테크코퍼레이션은 올해 8조 3천억 원의 매출을 달성했으며, 이는 전년 대비 1조 2천억 원 증가한 수치입니다. AI 부문은 2조 7천억 원의 매출을 기록했고, 클라우드 부문은 3,500억 원의 수익을 올렸습니다. 순이익은 1조 8천억 원을 기록했으며, 연구개발에 4,200억 원이 투자되었습니다. 회사는 평균 연봉 8천 5백만 원인 신규 직원 2만 5천 명을 채용했습니다.


### **4.3 Constraints**

Constraints are an important element that ensures output quality and consistency. In solar-pro2, clear constraints can guide the accurate fulfillment of complex requirements.

#### 4.3.1 Separating Positive and Negative Instructions

Clearly distinguish and present what should be done and what should not be done.

Below is a prompt example for newsletter generation tasks. When there are constraints that require output only in plain text rather than markdown output due to service environment requirements, positive and negative instructions can be separated as follows.


In [87]:
response = client.chat.completions.create(
   model="solar-pro2",
   messages=[
       {
           "role": "system",
           "content": """# Role
You are a newsletter writer creating content for email distribution.

# DO:
- Use clear headlines and numbered lists
- Only use plain text
- Keep sentences short and conversational

# DON'T:
- NEVER use **bold**, *italics*, or # headers
- NEVER use bullet points with -, *, or + symbols
- NEVER use emojis
- NEVER add markdown formatting"""
       },
       {
           "role": "user",
           "content": """Create a tech newsletter about AI industry trends."""
       }
   ],
   max_tokens=500,
   temperature=0.4,
   reasoning_effort="low"
)

print(response.choices[0].message.content)

Hi there,  

Here are five key trends shaping the AI industry this quarter:  

1. Generative AI expands into enterprise tools  
Companies are integrating generative AI into project management software, CRM platforms, and design applications to automate content creation and data analysis. Expect more niche tools targeting specific industries like legal or healthcare.  

2. Smaller, faster models challenge big AI  
Efficiency-focused models like Microsoft’s Phi-3 and Mistral’s 7B are gaining traction for their ability to run locally on consumer hardware while matching performance of larger systems in specialized tasks.  

3. AI regulation gains momentum  
The EU’s AI Act has entered final negotiations, with the US and UK also drafting sector-specific guidelines. Businesses should prepare for increased scrutiny around transparency and bias mitigation.  

4. Multimodal AI breaks into mainstream apps  
Tools combining text, image, and audio understanding—like OpenAI’s GPT-4o—are being embed

#### 4.3.2 Prioritized Constraints

Clearly establish priorities for requirements that may conflict when there are many instructions and constraints.

In complex tasks, situations arise where multiple goals are difficult to achieve simultaneously, so consistent decision-making criteria must be provided through a clear priority system. This helps the model make appropriate choices in dilemma situations and ensures quality and consistency of results.

**Importance Order Expression:**
- CRITICAL / HIGHEST / MOST IMPORTANT → HIGH → MEDIUM → LOW / LOWEST
- Priority 1 (Absolute) → Priority 2 (Important) → Priority 3 (Optional)
- Non-negotiable → Essential → Preferred → Optional


In [101]:
# 우선순위가 있는 제약사항 예제 - 수치 요약 with 마크다운 표
response = client.chat.completions.create(
   model="solar-pro2",
   messages=[
       {
           "role": "system",
           "content": """You are a data formatter converting text to structured tables.

## Formatting Guidelines

### PRIORITY 1: Only response in markdown table format (CRITICAL)
- NEVER add title of the table
- Do not add any explanation text

### PRIORITY 2: Table Format Compliance
- All numerical data MUST be in markdown table format
- Use exact syntax: | Header | Value | with proper alignment
- Include every single number from source text

### PRIORITY 3: Complete Coverage
- No information from source should be omitted
- Add category groupings if needed

Convert the provided text into structured table format."""
       },
       {
           "role": "user",
           "content": """GlobalTech reported exceptional Q4 performance with total revenue reaching $89.4 billion, representing 15.2% year-over-year growth. Operating income was $24.7 billion, up 18.5% from last year. Net income climbed to $19.3 billion, a 22.1% increase. The company's profit margin improved to 21.6%, while return on equity reached 28.9%. Cash flow from operations totaled $32.1 billion."""
       }
   ],
   max_tokens=4000,
   temperature=0.1,
   reasoning_effort="high"
)

print(response.choices[0].message.content)

| Metric | Value |
|--------|-------|
| Total Revenue | $89.4 billion |
| Total Revenue Growth | 15.2% |
| Operating Income | $24.7 billion |
| Operating Income Growth | 18.5% |
| Net Income | $19.3 billion |
| Net Income Growth | 22.1% |
| Profit Margin | 21.6% |
| Return on Equity | 28.9% |
| Cash Flow from Operations | $32.1 billion |


### **4.4 Output Format**

Definition of output format determines the usability and consistency of results. Expected output formats can be guided using structural templates and examples. Clear format specification facilitates subsequent processing or system integration and ensures consistent quality results.

#### 4.4.1 Presenting Structural Templates

Guide the parts that the model should respond to and fill in the expected output format using brackets. This method clearly conveys to the model the exact structure and the nature of content that should be included in each section, ensuring consistency and completeness of results. The descriptions within brackets specifically present the type and purpose of content to go in that position, helping the model generate appropriate information.

In [102]:
response = client.chat.completions.create(
    model="solar-pro2",
    messages=[
        {
            "role": "system",
            "content": """Create a brief business summary using this template:
## 🛠️ Issue: [main problem]
## 💡 Impact: [consequence]  
## ✅ Solution: [recommended action]

Keep each section to 1-2 sentences."""
        },
        {
            "role": "user",
            "content": """Situation: Website loading speed decreased by 40%, causing 15% drop in user engagement."""
        }
    ],
    max_tokens=800,
    temperature=0.5,
    reasoning_effort="low"
)

print(response.choices[0].message.content)


## 🛠️ Issue: Website loading speed has decreased by 40%, significantly slowing user access.  
## 💡 Impact: The slowdown has caused a 15% drop in user engagement, risking revenue and retention.  
## ✅ Solution: Optimize images, enable caching, and upgrade hosting to improve load times and recover engagement.


#### 4.4.2 JSON Output Format

Guide the required format and the parts that the model should fill in. It's important to instruct not to include any other text besides the output.
In particular, specify constraints like "Respond ONLY with the JSON object" to prevent additional explanations or comments from being included, and ensure output in pure data format that systems can process directly.


In [103]:
response = client.chat.completions.create(
    model="solar-pro2",
    messages=[
        {
            "role": "system",
            "content": """Extract key information and return as JSON only:
{
  "company": "company name",
  "revenue": "revenue amount",
  "growth": "growth percentage"
}

Return ONLY the JSON object."""
        },
        {
            "role": "user",
            "content": """News: TechCorp reported $2.5 billion revenue, up 18% from last year."""
        }
    ],
    max_tokens=200,
    temperature=0.3,
    reasoning_effort="low"
)

print(response.choices[0].message.content)


{
  "company": "TechCorp",
  "revenue": "$2.5 billion",
  "growth": "18%"
}


#### 4.4.3 Few-Shot Learning

Few-Shot Learning is a technique that teaches models desired output patterns by presenting a few specific examples. It's very effective as it can intuitively convey task requirements and expected results through actual input-output examples instead of long explanations. It teaches expected output patterns through concrete and practical examples, maximizing learning effects by presenting progressively from simple examples to complex situations.

**Core Principles of Few-Shot Learning:**

- **Progressive Complexity**: Starting with basic simple cases and gradually increasing difficulty to exceptional situations or complex problems to optimize the model's learning curve.

- **Type-based Classification**: Group examples by error or situation types to clearly teach the characteristics and processing methods of each category.

- **Practical Examples**: Use concrete and realistic situations that can occur in actual service environments as examples to increase the model's practicality.

- **Include Processing Guidelines**: Rather than simply presenting input-output examples, provide specific guidelines on how to approach and process each type.

Below is a simple proofreading prompt example. It divides guides for each error type that needs to be proofread and includes appropriate examples for each guide to help the model systematically recognize and correct various error types.


In [20]:
response = client.chat.completions.create(
  model="solar-pro2",
  messages=[
      {
          "role": "system",
          "content": """# Role
You are a professional English proofreader specializing in business and formal documents.

# Instructions
Correct English text based on error type.
Return only the corrected sentence without explanations.

# Grammar Errors
- Fix grammatical mistakes with proper structure
Input: "The report was wrote by our team yesterday."
Output: "The report was written by our team yesterday."

# Punctuation Errors
- Apply standard punctuation rules
Input: "Please send the proposal to John Sarah and Michael"
Output: "Please send the proposal to John, Sarah, and Michael."

# Sentence Structure Improvement
- Rewrite for clarity and natural flow
Input: "What we discussed in the meeting yesterday is something that needs immediate action."
Output: "The issues we discussed in yesterday's meeting require immediate action."

# Awkward Expression Correction
- Improve unnatural or unclear expressions
Input: "We need to do a discussion about the budget."
Output: "We need to discuss the budget."
"""
      },
      {
          "role": "user",
          "content": """Input: The presentation was gave to all department heads Tom Lisa and Mark yesterday and we need to do a review about it.
          Output: """
      }
  ],
  max_tokens=4000,
  temperature=0.2,
  reasoning_effort="high"
)

print(response.choices[0].message.content)

The presentation was given to all department heads, Tom, Lisa, and Mark yesterday, and we need to review it.


---

## **5. Emphasizing Critical Instructions**

Use special emphasis methods and emphasis words to focus the model's attention so that important rules or constraints are not ignored. Unlike general instructions, requirements that must be absolutely followed use visual emphasis and repetition.

**`Visual Emphasis Methods`:**
- **Use of Capital Letters**: Indicate importance with CRITICAL, MANDATORY, WARNING, IMPORTANT
- **Dividers and Boxes**: Use #, ---, === to separate important sections
- **Repetitive Emphasis**: Relocate key instructions to the beginning and end of prompts
- **Numbers and Checklists**: List essential requirements in order to prevent omissions

**`Emphasis Word Usage`:**
- **Absolute Compliance**: CRITICAL, MANDATORY, REQUIRED, MUST, NEVER, ALWAYS
- **Warning Indication**: WARNING, ALERT, CAUTION, FAILURE TO COMPLY
- **Priority**: PRIORITY, ESSENTIAL, VITAL, CRUCIAL, NON-NEGOTIABLE
- **Verification Induction**: VERIFY, CHECK, CONFIRM, VALIDATE, ENSURE

### **5.1 Self-Verification**

Self-verification is a technique that guides the model to check the accuracy and completeness of results by itself after completing tasks. Similar to how humans review after completing important work, this instructs the model to perform systematic verification procedures to detect and correct errors in advance. Provide specific checklists that allow the model to check essential task requirements one by one, and specify in the prompt to check compliance with each item.


In [ ]:
response = client.chat.completions.create(
   model="solar-pro2",
   messages=[
       {
           "role": "system",
           "content": """Convert USD amounts to multiple currencies using specified exchange rates.

# Exchange Rates (Use these exact rates)
- 1 USD = 1,320 KRW (Korean Won)
- 1 USD = 150 JPY (Japanese Yen) 
- 1 USD = 7.2 CNY (Chinese Yuan)

# Format Requirements
- USD: $ symbol with commas and 2 decimals
- KRW: ₩ symbol with commas, no decimals
- JPY: ¥ symbol with commas, no decimals
- CNY: ¥ symbol with commas and 2 decimals

# VERIFICATION CHECKLIST
Before submitting your response, verify:
1. USD amount × 1,320 = KRW calculation is correct
2. USD amount × 150 = JPY calculation is correct
3. USD amount × 7.2 = CNY calculation is correct
4. All currency symbols are properly used (₩, ¥, ¥, $)
5. KRW and JPY show no decimal places
6. USD and CNY show exactly 2 decimal places
7. All amounts include proper comma separators

CRITICAL WARNING: Failure to follow these formatting rules will result in unusable output for financial systems.

Convert the USD amount and return only the converted amounts without any explanations."""
       },
       {
           "role": "user",
           "content": """$2,850.75 USD"""
       }
   ],
   max_tokens=4000,
   temperature=0.1,
   reasoning_effort="high"
)

print(response.choices[0].message.content)

$2,850.75  
₩3,762,990  
¥427,613  
¥20,525.40


### **5.2 Context Management**

#### 5.2.1 Key Information Placement in Long Contexts

Ensure important instructions are not ignored through key information placement in long contexts. Important instructions can be continuously reminded during the information processing by placing them at the beginning and end of prompts.
The prompt below is an example of a RAG system that accurately extracts only numerical data from a long financial document of over 9000 characters stored in the `{DOCUMENT}` variable.

First, set up the document to be used as context.


In [8]:
# 샘플 문서
DOCUMENT = """
## TechCorp 2024 Annual Financial Report

### Executive Summary
TechCorp achieved record-breaking financial performance in 2024, with total revenue reaching $34.7 billion, representing a 28% year-over-year increase from $27.1 billion in 2023. The company's strategic focus on artificial intelligence, quantum computing, and cloud services drove exceptional growth across all business segments. Net income surged to $5.8 billion, up 42% from the previous year, while operating cash flow reached $8.2 billion.

### Historical Performance Analysis
Over the past five years, TechCorp has consistently outperformed industry benchmarks. Revenue growth averaged 24% annually since 2019, when the company reported $14.2 billion in revenue. The transformation from a traditional hardware manufacturer to a comprehensive technology solutions provider has been remarkable. In 2020, during the pandemic, TechCorp pivoted quickly to remote collaboration tools, which contributed to a 31% revenue spike that year.

### Revenue Breakdown by Division

**Software Division**: Generated $12.8 billion in revenue (37% of total), up 35% from 2023's $9.5 billion. The division launched 14 new AI-powered products, including the revolutionary TechBot enterprise assistant and advanced cybersecurity suite. International expansion contributed 40% of software revenue, with particularly strong performance in Asia-Pacific markets growing 58% year-over-year.

**Hardware Division**: Contributed $10.2 billion (29% of total), showing 22% growth from $8.4 billion in 2023. Quantum computing processors became a major revenue driver, generating $2.1 billion alone. Edge computing devices and IoT sensors contributed $3.8 billion, while traditional server hardware accounted for $4.3 billion. Supply chain optimization reduced manufacturing costs by 12%.

**Cloud Services**: Delivered $7.8 billion (22% of total), with remarkable 41% year-over-year growth from $5.5 billion in 2023. Enterprise customers increased by 2,840 new accounts, bringing total enterprise clients to 47,300. Data storage services grew 48%, while AI-as-a-Service offerings expanded 65%. Multi-cloud management tools contributed $1.9 billion in revenue.

**Consulting Services**: Accounted for $3.9 billion (12% of total), growing 18% from $3.3 billion in 2023. Digital transformation projects represented 72% of consulting revenue at $2.8 billion. Cybersecurity consulting expanded rapidly, contributing $890 million. The division completed 1,247 major projects across 67 countries.

### Quarterly Performance Breakdown

**Q1 2024**: Revenue of $8.1 billion, growth of 24% YoY. Software led with $3.0 billion, while cloud services reached $1.8 billion. Operating margin was 21.2%.

**Q2 2024**: Revenue climbed to $8.6 billion, 26% growth. Hardware division peaked at $2.7 billion due to quantum processor launches. Net income hit $1.4 billion.

**Q3 2024**: Strong performance with $8.7 billion revenue, 29% growth. Cloud services surged to $2.0 billion. International sales comprised 54% of total revenue.

**Q4 2024**: Record quarterly revenue of $9.3 billion, 32% growth. All divisions exceeded projections. Operating cash flow reached $2.3 billion for the quarter.

### Profitability and Efficiency Metrics
Gross margin improved significantly to 71.4%, up from 67.8% in 2023, driven by higher-margin software and cloud services. Operating margin reached 24.1%, compared to 21.3% in the previous year. Return on equity (ROE) improved to 18.7%, while return on assets (ROA) reached 12.4%. Working capital efficiency improved by 15%, with inventory turnover increasing to 8.2 times annually.

EBITDA margin expanded to 28.9%, reflecting strong operational leverage. The company maintained healthy liquidity with $4.1 billion in cash and cash equivalents, plus $2.8 billion in short-term investments. Debt-to-equity ratio remained conservative at 0.34, providing financial flexibility for future investments.

### Operational Highlights and Workforce
Employee headcount grew to 127,800 global employees, with 18,900 new hires in 2024. The engineering team expanded by 8,400 professionals, while sales and marketing added 4,200 personnel. Employee satisfaction scores reached 87%, up from 82% in 2023. The company invested $340 million in employee training and development programs.

R&D investment totaled $4.7 billion (13.5% of revenue), focusing on quantum computing, artificial intelligence, cybersecurity, and sustainable technologies. The company filed 1,247 new patents in 2024, bringing the total patent portfolio to 8,934. Research facilities expanded to 23 locations across 12 countries.

### Technology and Innovation Initiatives
TechCorp's quantum computing division achieved several breakthroughs, including a 1000-qubit processor and room-temperature quantum storage. The AI research lab developed advanced large language models with 500 billion parameters. Sustainability initiatives included carbon-neutral data centers and 100% renewable energy usage by 2026.

Strategic partnerships expanded to include collaborations with 47 universities and 23 government research institutions. The company established innovation hubs in Singapore, Tel Aviv, and Stockholm. Venture capital arm invested $280 million in 34 early-stage technology startups.

### Market Position and Competitive Landscape
TechCorp maintains its position as the second-largest enterprise software provider globally, behind Microsoft but ahead of Oracle and SAP. In cloud infrastructure, the company ranks fourth globally with 12.3% market share, trailing Amazon Web Services, Microsoft Azure, and Google Cloud Platform.

The quantum computing market leadership position strengthened with 34% market share, ahead of IBM's 28% and Google's 19%. In cybersecurity solutions, TechCorp ranks third with 11.2% market share in the enterprise segment.

### Geographic Revenue Distribution
North America contributed 42% of revenue ($14.6 billion), growing 24% year-over-year. Europe, Middle East, and Africa (EMEA) generated 31% ($10.7 billion) with 28% growth. Asia-Pacific delivered exceptional performance with 27% of revenue ($9.4 billion) and 39% growth, driven by expansion in India, Japan, and Southeast Asia.

### Future Outlook and Strategic Initiatives
Management forecasts 2025 revenue of $43-46 billion, representing 24-33% growth. Software division expected to reach $16-17 billion, while cloud services projected to achieve $11-12 billion. The company plans to invest $6.2 billion in R&D, focusing on artificial general intelligence (AGI) research and quantum-classical computing integration.

Capital expenditures planned at $3.8 billion for 2025, including new data centers, manufacturing facilities, and research labs. The company targets 40% of revenue from AI-related products by 2027, up from 28% in 2024. International expansion will focus on Latin America and emerging Asian markets.

### Risk Factors and Mitigation Strategies
Key challenges include intensified competition in AI from Google, OpenAI, and Meta, potential regulatory changes affecting data privacy and AI development, ongoing geopolitical tensions affecting global operations, and talent acquisition difficulties in specialized fields like quantum computing and AI research.

Cybersecurity threats increased 67% in 2024, requiring enhanced security investments. Supply chain vulnerabilities persist for semiconductor components, leading to strategic stockpiling and supplier diversification. Climate change risks to data center operations prompted investments in resilient infrastructure and renewable energy sources.

The company allocated $890 million for risk mitigation, including $340 million for supply chain diversification, $280 million for cybersecurity enhancements, and $270 million for regulatory compliance and legal contingencies.

### Environmental, Social, and Governance (ESG)
TechCorp achieved carbon neutrality for Scope 1 and 2 emissions in 2024, two years ahead of schedule. Renewable energy now powers 94% of operations. Water usage decreased 23% through efficiency improvements. The company recycled 89% of electronic waste through certified programs.

Board diversity improved with 40% women and 30% underrepresented minorities. Executive compensation aligned with ESG targets, with 25% of bonuses tied to sustainability metrics. Community investment reached $47 million, supporting STEM education and digital literacy programs in underserved communities.

### Financial Statements Summary
Balance sheet strength improved with total assets of $48.3 billion, up from $39.7 billion in 2023. Shareholders' equity reached $31.2 billion. Long-term debt decreased to $8.9 billion from $10.1 billion. Book value per share increased to $89.34, representing 21% growth.

Cash flow from operations of $8.2 billion provided strong funding for growth investments. Free cash flow reached $5.4 billion after capital expenditures of $2.8 billion. The company returned $2.1 billion to shareholders through $1.3 billion in dividends and $800 million in share repurchases.
"""

Place the core constraints "extract numbers only" and "source citation required" at the beginning and end of the prompt so that instructions can be followed well even in long document contexts.

In [120]:
response = client.chat.completions.create(
   model="solar-pro2",
   messages=[
       {
           "role": "system",
           "content": f"""=== CRITICAL: Extract Numbers Only ===

# Role  
You are a financial data extraction specialist.

# Context Documents
<document>
{DOCUMENT}
</document>

# Instructions
Extract ONLY numerical data from the provided document. Return numbers with their units but NO explanatory text.

# MANDATORY Format
- Use this exact format: Revenue: $34.7 billion (Source: Executive Summary)
- Include dollar amounts, percentages, and growth rates ONLY
- No sentences, descriptions, or analysis
- One number per line

=== REMINDER: Numbers Only ===
Extract financial figures without any explanatory text or analysis."""
       },
       {
           "role": "user", 
           "content": """Extract TechCorp's 2024 total revenue, growth rate, and revenue breakdown by division with their growth rates."""
       }
   ],
   max_tokens=8000,
   temperature=0.1,
   reasoning_effort="high"
)

print(response.choices[0].message.content)

Revenue: $34.7 billion (Source: Executive Summary)  
Growth Rate: 28% (Source: Executive Summary)  
Software Division Revenue: $12.8 billion (Source: Revenue Breakdown by Division)  
Software Division Growth Rate: 35% (Source: Revenue Breakdown by Division)  
Hardware Division Revenue: $10.2 billion (Source: Revenue Breakdown by Division)  
Hardware Division Growth Rate: 22% (Source: Revenue Breakdown by Division)  
Cloud Services Revenue: $7.8 billion (Source: Revenue Breakdown by Division)  
Cloud Services Growth Rate: 41% (Source: Revenue Breakdown by Division)  
Consulting Services Revenue: $3.9 billion (Source: Revenue Breakdown by Division)  
Consulting Services Growth Rate: 18% (Source: Revenue Breakdown by Division)


---

## **6. Common Issues and Solutions**

### **6.1 Ambiguous Responses**

**❌ Problem:** Unclear or inconsistent answers

This is a problem situation where unclear analysis criteria in prompts cause the model to provide inconsistent and ambiguous responses. The system prompt simply provides abstract instructions like "analyze data and provide insights," and the user prompt also asks vague questions like "what do you think about our sales" without specific context or analysis criteria. As a result, the model doesn't know from what perspective to analyze or what criteria to evaluate, leading to different responses each time for the same data or providing non-specific, generic answers.

In [21]:
response = client.chat.completions.create(
   model="solar-pro2",
   messages=[
       {
           "role": "system",
           "content": """Analyze the data and provide insights in three sentences."""
       },
       {
           "role": "user",
           "content": """What do you think about our sales? Revenue: $500K, customers: 200, churn: 5%"""
       }
   ],
   max_tokens=400,
   temperature=0.5,
   reasoning_effort="low"
)

print(response.choices[0].message.content)

With a revenue of $500K from 200 customers, the average revenue per customer is $2,500, indicating a strong per-customer value. A churn rate of 5% is relatively low, suggesting good customer retention and stability in recurring revenue. To grow, consider strategies to increase customer acquisition or upselling to existing customers while maintaining the low churn.


**✅ Solution:** Provide specific instructions and sufficient context

Ambiguous responses occur when the model cannot accurately understand user requirements due to insufficient instructions. The key is to provide specific and detailed instructions for the tasks the model should perform.
It's also important to provide sufficient context to the model. Rather than simply presenting data, include background information necessary for the model to perform tasks, such as business situations, industry standards, and goals.

In [22]:
response = client.chat.completions.create(
    model="solar-pro2",
    messages=[
        {
            "role": "system",
            "content": """# Role: Sales Performance Analyst

# Required Analysis (exactly 3 sentences):
1. **Revenue per Customer**: Calculate and assess ($500K ÷ 200 customers)
2. **Churn Impact**: Evaluate 5% churn rate and monthly revenue loss
3. **Priority Action**: Recommend most critical improvement with specific target

Use specific numbers and clear recommendations."""
        },
        {
            "role": "user",
            "content": """## Current Metrics
- Monthly revenue: $500K
- Active customers: 200
- Monthly churn rate: 5%

## Business Context
- Industry: B2B SaaS (project management tools)
- Company stage: Series A startup
- Target: $1M monthly revenue by year-end

## Question
What's your performance assessment with specific recommendations?"""
        }
    ],
    max_tokens=400,
    temperature=0.3,
    reasoning_effort="low"
)

print(response.choices[0].message.content)

1. **Revenue per Customer**: The current revenue per customer is **$2,500** ($500K ÷ 200 customers), which is a baseline metric for assessing pricing and upsell opportunities.  
2. **Churn Impact**: A 5% monthly churn rate equates to losing **10 customers/month** (5% of 200) and **$25K in monthly recurring revenue** ($2,500 × 10), severely hindering growth toward the $1M target.  
3. **Priority Action**: **Reduce churn to 3% or below** (saving ~$15K/month) by implementing a customer success program focused on onboarding and proactive support, while simultaneously pursuing upsells to increase revenue per customer by **10–15%** to accelerate progress toward the $1M goal.


### **6.2 Inconsistent Output Format**

**❌ Problem:** Output different from requested format

This is a problem where the model responds in plain text or other formats despite requesting output in JSON format or specific structure. Accurate format is essential especially in API integration, data processing, and automation systems, but ambiguous format instructions can cause parsing errors or system failures. For example, cases where you request "respond in JSON" but the model outputs with explanatory text, or responds in a different form from the requested key structure.


In [143]:
response = client.chat.completions.create(
   model="solar-pro2",
   messages=[
       {
           "role": "system",
           "content": """Return the analysis in JSON format"""
       },
       {
           "role": "user",
           "content": """Analyze customer feedback: "Great product but expensive pricing" - Rating: 4/5"""
       }
   ],
   max_tokens=300,
   temperature=0.7,
   reasoning_effort="low"
)

print(response.choices[0].message.content)

Here is the analysis of the customer feedback in JSON format:

```json
{
  "feedback": "Great product but expensive pricing",
  "rating": {
    "score": 4,
    "out_of": 5,
    "sentiment": "positive"
  },
  "aspects": [
    {
      "aspect": "Product Quality",
      "sentiment": "positive",
      "confidence": 0.95,
      "keywords": ["Great product"]
    },
    {
      "aspect": "Pricing",
      "sentiment": "negative",
      "confidence": 0.85,
      "keywords": ["expensive pricing"]
    }
  ],
  "overall_sentiment": "mixed",
  "recommendation": "Consider addressing pricing concerns while maintaining product quality to improve overall satisfaction"
}
``` 

### Breakdown of the analysis:
1. **Overall Sentiment**: Mixed (positive product experience but negative pricing sentiment)
2. **Product Quality**: Strongly positive ("Great product")
3. **Pricing**: Negative concern ("expensive pricing")
4. **Rating**: 4/5 indicates high satisfaction despite one major drawback
5. **Actionable Ins

**✅ Solution:** Provide accurate structure specifications and concrete examples

Format inconsistencies often occur when output requirements are unclear or examples are insufficient. Rather than simply saying "respond in JSON," clearly define key names, data types, required/optional fields, and specify constraints such as prohibiting additional explanatory text.

Simply providing an `Output format example` can convey clear guidelines about exact output structure, key order, and value types to the model, greatly improving consistency. You can also set temperature low to require more strict format compliance.

In [168]:
response = client.chat.completions.create(
   model="solar-pro2",
   messages=[
       {
           "role": "system",
           "content": """Return the analysis in JSON format without any explanation.
Return ONLY a valid JSON object with these EXACT keys:
{
"sentiment_score": number (1-5 scale),
"sentiment_label": string ("positive" or "negative" or "neutral"),
"main_topics": array of strings (max 3 items),
"improvement_area": string (single most critical issue),
"confidence_level": number (0.0-1.0)
}
#Output Format Example
{"sentiment_score":3,"sentiment_label":"neutral","main_topics":["Service Speed","Staff Attitude"],"improvement_area":"Response Time","confidence_level":0.78}"""
       },
       {
           "role": "user",
           "content": """Analyze customer feedback: "Great product but expensive pricing" - Rating: 4/5"""
       }
   ],
   max_tokens=300,
   temperature=0.3,
   reasoning_effort="low"
)

print(response.choices[0].message.content)

{"sentiment_score":4,"sentiment_label":"positive","main_topics":["Product Quality","Pricing"],"improvement_area":"Pricing","confidence_level":0.92}


### **6.3 Instruction Non-compliance**

**❌ Problem:** Incomplete answers with missing requested components or essential sections

Instruction non-compliance occurs when some requirements are overlooked in complex requests or when there's no systematic verification process. In the academic writing conversion example below, the model is prone to performing only some rules because various editing rules must be applied simultaneously.

In [25]:
response = client.chat.completions.create(
  model="solar-pro2",
  messages=[
      {
          "role": "system",
          "content": """Convert text to academic writing style: use active voice, remove first-person pronouns, replace informal terms with precise academic vocabulary, and maintain formal tone without any explanation"""
      },
      {
          "role": "user",
          "content": """We think that the results show there's a clear connection between the variables. It's obvious that our hypothesis was right and we've proven our point. The data was analyzed by us and it shows that we're correct about this relationship."""
      }
  ],
  max_tokens=4000,
  temperature=0.1,
  reasoning_effort="high"
)

print(response.choices[0].message.content)

The results demonstrate a significant correlation between the variables. The findings support the initial hypothesis, validating the proposed relationship. Analysis of the data reveals a robust association, thereby substantiating the hypothesized connection. The empirical evidence aligns with theoretical expectations, confirming the validity of the posited framework.  

(Note: The response adheres to academic conventions by eliminating first-person pronouns, employing active voice where appropriate, and using precise terminology. The tone remains formal and objective, avoiding colloquial expressions.)


**✅ Solution:** Include work order and verification procedures

For effective resolution, each academic writing task should be clearly distinguished and numbered, and specific conversion examples should be provided to prevent confusion. Also, use checklists to guide the model to self-verify whether all editing rules have been applied.

In [23]:
response = client.chat.completions.create(
 model="solar-pro2",
 messages=[
     {
         "role": "system",
         "content": """# MANDATORY Academic Writing Requirements

Apply ALL 4 editing rules to the text:

## REQUIRED EDITING TASKS (ALL must be applied):
1. **Voice Conversion**: Change all passive voice to active voice throughout
2. **Pronoun Elimination**: Remove all first-person pronouns (I, we, our, us) and rewrite objectively
3. **Vocabulary Precision**: Replace ALL informal/vague terms with specific academic language:
 - think → conclude/demonstrate/indicate
 - show → reveal/demonstrate/establish
 - connection → correlation/relationship
 - clear → evident/apparent
 - right → accurate/valid
4. **Formality**: Convert contractions and casual language to formal academic standards

# VERIFICATION CHECKLIST
Before submitting, confirm ALL applied:
□ All passive voice converted to active voice
□ All first-person pronouns eliminated
□ All informal terms replaced with academic vocabulary
□ Formal academic tone maintained
□ Only edited text provided

## OUTPUT FORMAT:
[Only response with edited text]

CRITICAL: Never include any explanation"""
     },
     {
         "role": "user",
         "content": """We think that the results show there's a clear connection between the variables. It's obvious that our hypothesis was right and we've proven our point. The data was analyzed by us and it shows that we're correct about this relationship."""
     }
 ],
 max_tokens=4000,
 temperature=0.1,
 reasoning_effort="high"
)

print(response.choices[0].message.content)

The results indicate an evident correlation between the variables. The hypothesis is supported by the data, which demonstrates the validity of the proposed relationship. Analysis of the data reveals a statistically significant relationship, validating the initial hypothesis.


### **6.4 Hallucination**

**❌ Problem:** Providing facts beyond given information or providing incorrect information

This is a problem where the model presents definitive causes based on speculation or asserts specific solutions in chatbot responses to customer inquiries, even when clear causes cannot be identified. For example, when there's insufficient information to respond to feedback like "the app crashes when uploading photos," the model definitively presents specific causes and asserts arbitrary solutions, which can give customers false expectations or lead to inappropriate solution attempts.

In [187]:
response = client.chat.completions.create(
   model="solar-pro2",
   messages=[
       {
           "role": "system",
           "content": """You are a customer support chatbot for a mobile app.

# Known Technical Issues Context:
- Login failures often occur due to incorrect credentials, expired sessions, browser cache issues, server maintenance, or account lockouts
- Typical solutions include password reset, cache clearing, trying different browsers, or checking account status

Provide helpful troubleshooting advice based on the customer's issue."""
       },
       {
           "role": "user",
           "content": """Customer message: "The app keeps crashing every time I try to upload a photo. What's wrong?"."""
       }
   ],
   max_tokens=4000,
   temperature=0.3,
   reasoning_effort="high"
)

print(response.choices[0].message.content)

I'm sorry to hear you're experiencing this issue! Let's troubleshoot the photo upload crash step by step:  

1. **Check App Updates**  
   Ensure you're using the latest app version. Outdated versions may have unresolved bugs—update via your device's app store.  

2. **Free Up Storage Space**  
   Low storage can cause crashes during uploads. Delete unnecessary files or apps to free up space, then try again.  

3. **Try a Smaller/Simpler Photo**  
   Large files or high-resolution images may overwhelm the app. Test with a smaller photo or compressed file (e.g., JPEG instead of RAW).  

4. **Clear App Cache**  
   - Go to **Settings > Apps > [Your App Name] > Storage > Clear Cache**.  
   - If the issue persists, try **Clear Data** (note: this will log you out).  

5. **Restart Your Device**  
   Background processes might interfere. Restart your phone/tablet and reopen the app.  

6. **Check Internet Connection**  
   A unstable connection during uploads can trigger crashes. Ensure you

**✅ Solution:** Making it admit what it doesn't know

Hallucination occurs when the model draws uncertain conclusions based on limited information or generates non-existent details. In customer support chatbots, providing incorrect information can particularly lead to customer dissatisfaction, requiring a cautious approach. For effective resolution, you should provide instructions to explicitly acknowledge uncertainty if exact causes cannot be identified. Guide the model to use qualified expressions like `I'm not sure of the exact cause` when it cannot answer within the given context.

In [188]:
response = client.chat.completions.create(
   model="solar-pro2",
   messages=[
       {
           "role": "system",
           "content": """You are a customer support chatbot for an e-commerce platform.

# Known Technical Issues Context:
- Login failures often occur due to incorrect credentials, expired sessions, browser cache issues, server maintenance, or account lockouts
- Typical solutions include password reset, cache clearing, trying different browsers, or checking account status

# VERIFICATION BEFORE RESPONDING:
- Can I identify the specific cause from the customer's message? If NO → ONLY respond "Sorry, I don't know the exact cause."
- Am I about to guess or assume? If YES → ONLY respond "Sorry, I don't know the exact cause."

IMPORTANT: When the cause is unclear, admit uncertainty rather than guessing."""
       },
       {
           "role": "user",
           "content": """Customer message: "The app keeps crashing every time I try to upload a photo. What's wrong?"."""
       }
   ],
   max_tokens=4000,
   temperature=0.3,
   reasoning_effort="high"
)

print(response.choices[0].message.content)

Sorry, I don't know the exact cause.


---

## **7. Conclusion**

solar-pro2 is a model that is highly responsive to clear and structured prompts. By systematically designing prompts following the principles in this guide, you can maximize the model's excellent performance.

The key is clarity, structure, and specificity. Provide accurate instructions without ambiguity, compose prompts with logical structure, and include specific examples and verification steps to achieve consistent, high-quality results.

We recommend developing prompts optimized for your specific use cases through continuous testing and improvement.